<a href="https://colab.research.google.com/github/theresaxx/Financial-Disclosure-NLP-10K/blob/main/4_2_finetuning_transformer_mda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set up of the notebook

In [ ]:
#Declare whether you are on colab or local
colab = True

In [ ]:
if colab==True:
  
  #mount drive
  from google.colab import drive
  drive.mount('/content/drive')


  #set path to folder in Google Drive
  data_path = "/content/drive/MyDrive/Masterthesis/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### import + load libraries

In [ ]:
#install packages
!pip install torch transformers memory_profiler datasets -q
!pip install transformers datasets evaluate -q
!pip install ray -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 119.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 49.6 MB/s eta 0:00:00


In [ ]:
#import libraries 
import pandas as pd
import numpy as np
import math

import torch
from torch import cuda
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, LongformerTokenizerFast, LongformerForSequenceClassification, LongformerConfig, LongformerModel, AdamW
from datasets import load_metric, Dataset
import evaluate
%load_ext memory_profiler
from sklearn.model_selection import train_test_split

#from sentence_transformers import SentenceTransformer, util
#from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator, TripletEvaluator, LabelAccuracyEvaluator

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

import random


In [ ]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf
 
      tf.random.set_seed(seed)
 
set_seed(42)

### Set up dataframe

In [ ]:
#Load dataframe
df = pd.read_csv(data_path+'data/train_sets/mda_small.csv', index_col=0)
df.tail()

,mda,label
909,Item 7: Management's Discussion and Analysis o...,1
1464,Item 7. Management’s Discussion and Analysis o...,1
786,"Item 7, Management’s Discussion and Analysis o...",0
367,Item 7. Management's Discussion and Analysis o...,0
2062,ITEM 7.MANAGEMENT’S DISCUSSION AND ANALYSIS OF...,0


In [ ]:
df.shape

(762, 2)

In [ ]:
#convert dataframe to even label distribution 
#label_0 = df[df["label"] == 0]
#label_0 = label_0[:381]
#label_1 = df[df["label"] == 1]

In [ ]:
#concat to one df
#df = pd.concat([label_0, label_1], axis=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762 entries, 1565 to 2062
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mda     762 non-null    object
 1   label   762 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 17.9+ KB


In [ ]:
df.describe()

,label
count,762.000000
mean,0.500000
std,0.500328
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [ ]:
#label 1: low_performance
#label 0: performance

In [ ]:
#check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA A100-SXM4-40GB'

### Fine-tuning

Based on https://huggingface.co/docs/transformers/tasks/sequence_classification

Using PyTorch tensors

#### build dataframes + functions

In [ ]:
#define train, val, and test set
def df_split(df):

  df_train, df_val, = train_test_split(df, stratify=df['label'], test_size=0.30, random_state=42)
  df_val, df_test = train_test_split(df_val, stratify=df_val['label'],test_size=0.5, random_state=42)
  print(df_train.shape, df_test.shape, df_val.shape)

  #load data in torch dataframe
  dataset_train = Dataset.from_pandas(df_train)
  dataset_val = Dataset.from_pandas(df_val)
  dataset_test = Dataset.from_pandas(df_test)

  return dataset_train, dataset_val, dataset_test

In [ ]:
#metric function
def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    #argmax(preds.predictions, axis=1)
    #preds.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def TextClassification_with_Transformer(dataset_train, dataset_val, dataset_test, model_name, num_epochs, lr, out_path, log_path): 
                                        
  #define tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  #call Model, ForSequenceClassification-Models are best for classification tasks, instantiate GPU
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2, return_dict=False)

  #define a function for tokenization
  def tokenization(batched_text):
    return tokenizer(batched_text['mda'], padding = True, truncation=True, max_length = 512)   #add_special_tokens = True,

  #convert dataset to pytorch format
  df_train = dataset_train.map(tokenization, batched = True, batch_size = 32)
  df_val = dataset_val.map(tokenization, batched = True, batch_size = 32)
  df_test = dataset_test.map(tokenization, batched = True, batch_size = 32)

  df_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
  df_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
  df_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

  #metric function
  def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

  #define arguments for training  
  training_args = TrainingArguments(
      output_dir=out_path,                   #output directory
      num_train_epochs=num_epochs,          
      learning_rate=lr,      
      warmup_steps=500,                      #number of warmup steps for learning rate scheduler
      weight_decay=0.1,                      #strength of weight decay
      logging_dir=log_path,                  #directory for logs
      load_best_model_at_end=True,           #load the best model when finished training
      per_device_train_batch_size=32,        
      per_device_eval_batch_size=32,         
      logging_steps=200,                     #log & save weights each logging_steps
      save_steps=200,
      metric_for_best_model = "f1",          #select the base metrics
      evaluation_strategy="steps",           #evaluate each `logging_steps`
  ) 

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train,
    eval_dataset=df_val,
    compute_metrics=compute_metrics,
  )
  
  #train model
  trainer.train()
  
  #call summary
  trainer.evaluate()

  return trainer, model, df_test

In [ ]:
df_train, df_val, df_test = df_split(df)

(533, 2) (115, 2) (114, 2)


In [ ]:
#df_train[0]

#### Experiments 1: RoBERTa

##### apply model

In [ ]:
trainer_rb, model_rb, df_test_rb= TextClassification_with_Transformer(model_name = "roberta-base", 
                                                                 dataset_train=df_train,
                                                                 dataset_val=df_val,
                                                                 dataset_test = df_test,
                                                                 lr = 2.0e-5,           #1e-5, 
                                                                 num_epochs = 20,
                                                                 out_path= data_path+'models/mda/model1/out',
                                                                 log_path= data_path+'models/mda/model1/log')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

using `logging_steps` to initialize `eval_steps` to 200
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num example

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,0.690100,0.702767,0.500000,0.512821,0.500000,0.526316


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 114
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/Masterthesis/models/mda/model1/out/checkpoint-200
Configuration saved in /content/drive/MyDrive/Masterthesis/models/mda/model1/out/checkpoint-200/config.json
Model weights saved in /content/drive/MyDrive/Masterthesis/models/mda/model1/out/checkpoint-200/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/MyDrive/Masterthesis/models/mda/model1/out/checkpoint-200 (score: 0.5128205128205129).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSe

##### test and save model

In [ ]:
#evaluate on test set
model_rb.eval()
trainer_rb.predict(df_test_rb).metrics

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 115
  Batch size = 32


{'test_loss': 0.6869164109230042,
 'test_accuracy': 0.5565217391304348,
 'test_f1': 0.5405405405405405,
 'test_precision': 0.5555555555555556,
 'test_recall': 0.5263157894736842,
 'test_runtime': 0.8311,
 'test_samples_per_second': 138.37,
 'test_steps_per_second': 4.813}

In [ ]:
trainer_rb.save_model(data_path+"models/mda/model1/roberta")

Saving model checkpoint to /content/drive/MyDrive/Masterthesis/models/mda/model1/roberta
Configuration saved in /content/drive/MyDrive/Masterthesis/models/mda/model1/roberta/config.json
Model weights saved in /content/drive/MyDrive/Masterthesis/models/mda/model1/roberta/pytorch_model.bin


#### Experiments 2: SBERT

##### apply model

In [ ]:
trainer_sb, model_sb, df_test_sb= TextClassification_with_Transformer(model_name ="sentence-transformers/all-mpnet-base-v2", 
                                                                 dataset_train=df_train,
                                                                 dataset_val=df_val,
                                                                 dataset_test = df_test,
                                                                 num_epochs = 20,
                                                                 lr = 2.0e-5,   #1e-5,
                                                                 out_path= data_path+'models/mda/model2/out',
                                                                 log_path= data_path+'models/mda/model2/log')

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/co

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

using `logging_steps` to initialize `eval_steps` to 200
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,0.677200,0.735876,0.464912,0.573427,0.476744,0.719298


The following columns in the evaluation set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 114
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/Masterthesis/models/mda/model2/out/checkpoint-200
Configuration saved in /content/drive/MyDrive/Masterthesis/models/mda/model2/out/checkpoint-200/config.json
Model weights saved in /content/drive/MyDrive/Masterthesis/models/mda/model2/out/checkpoint-200/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/MyDrive/Masterthesis/models/mda/model2/out/checkpoint-200 (score: 0.5734265734265734).
The following columns in the evaluation set don't have a corresponding argument in `MPNetForSequence

##### test and save model

In [ ]:
#evaluate on test set
model_sb.eval()
trainer_sb.predict(df_test_sb).metrics

The following columns in the test set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 115
  Batch size = 32


{'test_loss': 0.6823186278343201,
 'test_accuracy': 0.6,
 'test_f1': 0.6406249999999999,
 'test_precision': 0.5774647887323944,
 'test_recall': 0.7192982456140351,
 'test_runtime': 0.8789,
 'test_samples_per_second': 130.848,
 'test_steps_per_second': 4.551}

In [ ]:
#only the best model of Sbert is saved
trainer_sb.save_model(data_path+"models/mda/model2/sbert")

Saving model checkpoint to /content/drive/MyDrive/Masterthesis/models/mda/model2/sbert
Configuration saved in /content/drive/MyDrive/Masterthesis/models/mda/model2/sbert/config.json
Model weights saved in /content/drive/MyDrive/Masterthesis/models/mda/model2/sbert/pytorch_model.bin


#### Experiment 3: FinBERT

In [ ]:
#https://github.com/yya518/FinBERT "yiyanghkust/finbert-pretrain"

##### apply model

In [ ]:
trainer_fb, model_fb, df_test_fb= TextClassification_with_Transformer(model_name = "yiyanghkust/finbert-pretrain", 
                                                                 dataset_train=df_train,
                                                                 dataset_val=df_val,
                                                                 dataset_test = df_test,
                                                                 num_epochs = 10,
                                                                 lr = 2.0e-5,      #2.0e-5,   #1e-5,
                                                                 out_path= data_path+'models/mda/model3/out',
                                                                 log_path= data_path+'models/mda/model3/log')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--yiyanghkust--finbert-pretrain/snapshots/88ab954a39ea6d3ce2b62cff086dd5ad1172c664/config.json
Model config BertConfig {
  "_name_or_path": "yiyanghkust/finbert-pretrain",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30873
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--yiyanghkust--

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

using `logging_steps` to initialize `eval_steps` to 200
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 53

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 114
  Batch size = 32


##### test and save model

In [ ]:
#evaluate on test set
model_fb.eval()
trainer_fb.predict(df_test_fb).metrics

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 115
  Batch size = 32


{'test_loss': 0.7100135087966919,
 'test_accuracy': 0.5130434782608696,
 'test_f1': 0.5483870967741935,
 'test_precision': 0.5074626865671642,
 'test_recall': 0.5964912280701754,
 'test_runtime': 0.8413,
 'test_samples_per_second': 136.693,
 'test_steps_per_second': 4.755}

In [ ]:
#only the best model of finbert is saved
trainer_fb.save_model(data_path+"models/mda/model3/finbert")

Saving model checkpoint to /content/drive/MyDrive/Masterthesis/models/mda/model3/finbert
Configuration saved in /content/drive/MyDrive/Masterthesis/models/mda/model3/finbert/config.json
Model weights saved in /content/drive/MyDrive/Masterthesis/models/mda/model3/finbert/pytorch_model.bin


#### Experiment 4: Longformer

##### define function

In [ ]:
def TextClassification_with_Longformer(dataset_train, dataset_val, dataset_test, model_name, num_epochs, lr, att_wind, out_path, log_path): 
                                        
  #define tokenizer
  model_lf = model_name  
  tokenizer_lf = LongformerTokenizerFast.from_pretrained(model_lf, max_length = 4096)

  #call Model
  model_lf = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", gradient_checkpointing=True, num_labels = 2, attention_window = att_wind).to('cuda')  

  #define a function for tokenization
  def tokenization_lf(batched_text):
      return tokenizer_lf(batched_text['mda'], padding = 'max_length', truncation=True, max_length = 4096)

  dataset_train_lf = df_train.map(tokenization_lf, batched = True, batch_size = 32)
  dataset_val_lf = df_val.map(tokenization_lf, batched = True, batch_size = 32)
  dataset_test_lf = df_test.map(tokenization_lf, batched = True, batch_size = 32) 

  dataset_train_lf.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
  dataset_val_lf.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
  dataset_test_lf.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  #metric function
  def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

  training_args = TrainingArguments(
      output_dir = out_path,
      num_train_epochs = num_epochs,
      per_device_train_batch_size = 32,
      per_device_eval_batch_size= 32,
      evaluation_strategy = "steps",
      learning_rate= lr, 
      gradient_accumulation_steps = int(math.ceil(len(dataset_train_lf) / 32) / 1) * 3, #  16, 
      disable_tqdm = False, 
      warmup_steps=200,
      weight_decay=0.1,
      logging_steps = 100,
      fp16 = True,
      logging_dir= log_path,
      load_best_model_at_end=True,
      metric_for_best_model = "f1",
      #dataloader_num_workers = 0,
  )

  #instantiate the trainer class and check for available devices
  trainer_lf = Trainer(
      model=model_lf,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=dataset_train_lf,
      eval_dataset=dataset_val_lf, 
  )

  #train model
  trainer_lf.train()  
  #call summary
  trainer_lf.evaluate()

  return trainer_lf, model_lf, dataset_test_lf

##### apply model

In [ ]:
trainer_lf, model_lf, df_test_lf= TextClassification_with_Longformer(model_name = "allenai/longformer-base-4096", 
                                                                 dataset_train=df_train,
                                                                 dataset_val=df_val,
                                                                 dataset_test=df_test,
                                                                 lr = 3e-5, # 5e-5 , #2.0e-5, #3e-5, 5e-5,10e-5
                                                                 num_epochs = 3,        #20
                                                                 att_wind = 512,           #256    #512    #1024
                                                                 out_path= data_path+'models/mda/model4/out',
                                                                 log_path= data_path+'models/mda/model4/log')

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/7a2d50a05513897192e79ea7c750493795d7cec0/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/7a2d50a05513897192e79ea7c750493795d7cec0/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/7a2d50a05513897192e79ea7c750493795d7cec0/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/7a2d50a05513897192e79ea7c750493795d7cec0/config.json
Model config LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn

Step,Training Loss,Validation Loss


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on C

Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...


##### test and save model

In [ ]:
#evaluate on text set
model_lf.eval()
trainer_lf.predict(df_test_lf).metrics

The following columns in the test set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: __index_level_0__, mda. If __index_level_0__, mda are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 115
  Batch size = 32
Initializing global attention on CLS token...


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...


{'test_loss': 0.6935759782791138,
 'test_accuracy': 0.5043478260869565,
 'test_f1': 0.658682634730539,
 'test_precision': 0.5,
 'test_recall': 0.9649122807017544,
 'test_runtime': 4.1689,
 'test_samples_per_second': 27.586,
 'test_steps_per_second': 0.959}

In [ ]:
#only the best model of Longformer is saved
trainer_lf.save_model(data_path+"models/mda/model4/longformer")

Saving model checkpoint to /content/drive/MyDrive/Masterthesis/models/mda/model4/longformer
Configuration saved in /content/drive/MyDrive/Masterthesis/models/mda/model4/longformer/config.json
Model weights saved in /content/drive/MyDrive/Masterthesis/models/mda/model4/longformer/pytorch_model.bin


### Backlog